In [15]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['portieri_23.xlsx', 'portieri_22.xlsx', 'portieri_21.xlsx', 'portieri_20.xlsx', 'portieri_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'portieri_23.xlsx': 2023,'portieri_22.xlsx': 2022,'portieri_21.xlsx': 2021,'portieri_20.xlsx': 2020,'portieri_19.xlsx': 2019}



In [16]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche
# suddiviso per annata

correlations = []
years = []

for file in files:
    
    data_frame = pd.read_excel(file)  # Leggi il file Excel

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Save%','CS%', 
    'Save%.1', 'Cmp%','Launch%', 'AvgLen','Launch%.1', 'AvgLen.1', 'Stp%'], inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")  # Ottengo in output il grafico di correlazione
    fig.show()

# Inserisco questa riga di codice in modo che in ogni momento si possano visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')


Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'GA', 'SoTA', 'Saves', 'CS', 'PKatt', 'PKA', 'PKsv', 'PKm', 'FK', 'CK', 'OG', 'Cmp', 'Att', 'Att (GK)', 'Thr', 'Att.1', 'Opp', 'Stp']


In [17]:
# Questa parte di codice serve per conoscere, se desiderato, quali sono le statistiche meglio e peggio correlate nel frame scelto
def best_worst(stat):
 for file in files:
    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    data_frame = pd.read_excel(file)  # Leggi il file Excel

    # Rimuovi le colonne che non vuoi usare
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Save%','CS%', 
    'Save%.1', 'Cmp%','Launch%', 'AvgLen','Launch%.1', 'AvgLen.1', 'Stp%'], inplace=True) 

    # Calcolo della matrice di correlazione
    corr_matrix = data_frame.corr()

    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset.")

stat = 'Age'
best_worst(stat)


   

Anno 2023: Le 5 statistiche meglio correlate con Age sono:
OG       0.270398
PKA      0.174924
PKatt    0.160464
PKm      0.158382
GA       0.138354
Name: Age, dtype: float64
Anno 2023: Le 5 statistiche peggio correlate con Age sono:
Thr     0.014828
FK     -0.003175
PKsv   -0.067236
Stp    -0.071400
Min    -0.163399
Name: Age, dtype: float64
Anno 2022: Le 5 statistiche meglio correlate con Age sono:
PKsv    -0.027938
PKA     -0.032219
FK      -0.043263
PKatt   -0.069601
OG      -0.070172
Name: Age, dtype: float64
Anno 2022: Le 5 statistiche peggio correlate con Age sono:
Min     -0.176647
Stp     -0.176916
CK      -0.186397
Saves   -0.194855
PKm     -0.277313
Name: Age, dtype: float64
Anno 2021: Le 5 statistiche meglio correlate con Age sono:
PKA       0.308248
PKatt     0.262189
Starts    0.235685
MP        0.233535
SoTA      0.231605
Name: Age, dtype: float64
Anno 2021: Le 5 statistiche peggio correlate con Age sono:
PKsv    0.048521
OG      0.047237
PKm     0.031403
Min    -0.04785

In [18]:
# Questa parte di codice serve per conoscere, se desiderato, la correlazione di due statistiche specifiche annata per annata
# Le statistiche di cui si vuole sapere la correlazione vanno riportate al posto di Stat1 e Stat2

def correlazione(stat1,stat2):

 correlations = []

 for file in files:
    # Leggi il file Excel
    data_frame = pd.read_excel(file)

    # Rimuovi le colonne che non sono utili per l'analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Save%','CS%', 
    'Save%.1', 'Cmp%','Launch%', 'AvgLen','Launch%.1', 'AvgLen.1', 'Stp%'], inplace=True)

    # Calcola la matrice di correlazione
    corr_matrix = data_frame.corr()

    # Ottieni l'anno dal file
    year = year_mapping[file]
    
    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")
    else:
        correlations.append((year, None))
        print(f"Anno {year}: Una o entrambe le statistiche ({stat1}, {stat2}) non sono presenti nel dataset.")

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2} nel tempo", markers=True)
    fig.update_xaxes(tickmode='linear', dtick=1)
    fig.show()
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide.")

stat1 = 'PKsv'
stat2 = 'Age'

correlazione(stat1,stat2)


Anno 2023: Correlazione tra PKsv e Age = -0.06723562467833986
Anno 2022: Correlazione tra PKsv e Age = -0.027937746591643442
Anno 2021: Correlazione tra PKsv e Age = 0.048520567469768516
Anno 2020: Correlazione tra PKsv e Age = 9.696809597111134e-05
Anno 2019: Correlazione tra PKsv e Age = -0.2297210796239639
